In [1]:
import pandas as pd

import os
from glob import glob

In [2]:
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

dataset = 'ouro_volume_agricola_lavouras'

In [3]:
arquivos_csv = []
arquivos_csv.extend(glob(os.path.join(raiz, 'ouro_volume_agricola_lavouras*.csv')))
arquivos_csv.sort()

tabelas = (
    'Perm Culturas',
    'Perm Municípios',
    'Perm UF',
    'Temp Culturas',
    'Temp Municípios',
    'Temp UF',
    'Total-CF'
)

In [4]:
perm_munic = (arquivos_csv[0], arquivos_csv[1])
tab_perm_munic = (tabelas[0], tabelas[1])

temp_munic = (arquivos_csv[3], arquivos_csv[4])
tab_temp_munic = (tabelas[3], tabelas[4])

csv_uf = (arquivos_csv[2], arquivos_csv[5], arquivos_csv[6])
tab_uf = (tabelas[2], tabelas[5], tabelas[6])

In [5]:
pd.read_csv(arquivos_csv[2])

,uf,ano,hectares
0,11,2003,244016
1,11,2004,214874
2,11,2005,223183
3,11,2006,210902
4,11,2007,210403
...,...,...,...
535,53,2018,1769
536,53,2019,1661
537,53,2020,1661
538,53,2021,1661


In [6]:
with pd.ExcelWriter('/home/eduardo/Downloads/lavouras.xlsx', mode='w', engine="openpyxl") as export:
    for arquivo, tabela in zip(perm_munic, tab_perm_munic):
        df = (pd.read_csv(arquivo)
              .merge(pd.read_parquet(f'{raiz}/prata_geocodigo_ibge.parquet')
                     .filter(['UF-id', 'UF-sigla', 'municipio-id', 'municipio-nome']),
                     how='left',
                     left_on='municipio',
                     right_on='municipio-id'
                    )
        )

        if 'cultura' in df.columns:
              df = df.pivot_table(
                       index=['uf','UF-sigla', 'municipio', 'municipio-nome', 'cultura'],
                       columns='ano',
                       values='hectares'
                       )
        else:
              df = df.pivot_table(
                       index=['uf','UF-sigla', 'municipio', 'municipio-nome'],
                       columns='ano',
                       values='hectares'
                      )

        df.reset_index().to_excel(export, sheet_name=tabela, index=False)
    
    
    for arquivo, tabela in zip(temp_munic, tab_temp_munic):
        df = (pd.read_csv(arquivo)
              .merge(pd.read_parquet(f'{raiz}/prata_geocodigo_ibge.parquet')
                     .filter(['UF-id', 'UF-sigla', 'municipio-id', 'municipio-nome']),
                     how='left',
                     left_on='municipio',
                     right_on='municipio-id'
                    )
        )

        if 'cultura' in df.columns:
              df = df.pivot_table(
                      index=['uf','UF-sigla', 'municipio', 'municipio-nome', 'cultura', 'safra'],
                      columns='ano',
                      values='valor'
                    )
        else:
              df = df.pivot_table(
                      index=['uf','UF-sigla', 'municipio', 'municipio-nome', 'safra'],
                      columns='ano',
                      values='valor'
                    )

        df.reset_index().to_excel(export, sheet_name=tabela, index=False)

In [7]:
with pd.ExcelWriter('/home/eduardo/Downloads/lavouras.xlsx', mode='a', engine="openpyxl", if_sheet_exists='replace') as export:
    for arquivo, tabela in zip(csv_uf, tab_uf):
        df = (pd.read_csv(arquivo)
              .merge(pd.read_parquet(f'{raiz}/prata_geocodigo_ibge.parquet')
                     .filter(['UF-id', 'UF-sigla']),
                     how='left',
                     left_on='uf',
                     right_on='UF-id'
                    )
        )

        if 'hectares' in df.columns:
              df = df.pivot_table(
                  index=['uf','UF-sigla'],
                  columns='ano',
                  values='hectares'
                  )
        elif 'safra' in df.columns:
             df = df.pivot_table(
                  index=['uf','UF-sigla', 'safra'],
                  columns='ano',
                  values='valor'
                  )
        else:
              df = df.pivot_table(
                  index=['uf','UF-sigla'],
                  columns='ano',
                  values=['valor', 'cf']
                  )

        df.reset_index().to_excel(export, sheet_name=tabela)